# Valorant Agent Picker Training

In this notebook, we wiil be using our 'Valorant Agent Picker' dataset to train a 'Random Forest Classifier' model and test its accuracy.

## Importing Dependencies

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

## Dataset Retreival

In [2]:
# GETTING THE DATASET FROM MY GITHUB REPOSITORY
!git clone https://github.com/omkanekar28/Valorant-Agent-Picker.git

Cloning into 'Valorant-Agent-Picker'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 86 (delta 32), reused 66 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 390.90 KiB | 3.69 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [3]:
dataset = pd.read_excel("/content/Valorant-Agent-Picker/data/dataset_complete.xlsx")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,Agent_Type,Playstyle,Difficulty,Ability_Preference,Gun_Type,Agent
0,Initiator,Info-gathering,Hard,Healing,SMGs,Skye
1,Controller,Balanced,Easy,Agility,Rifles,Brimstone
2,Controller,Aggressive,Hard,Agility,SMGs,Omen
3,Controller,Supportive,Easy,Smokes,Rifles,Harbor
4,Sentinel,Map-control,Easy,Agility,Rifles,Killjoy
...,...,...,...,...,...,...
995,Initiator,Balanced,Easy,Agility,Machine Guns,Sova
996,Sentinel,Aggressive,Easy,Smokes,Shotguns,Killjoy
997,Initiator,Info-gathering,Easy,Smokes,SMGs,Fade
998,Sentinel,Supportive,Hard,Healing,Rifles,Sage


## Dataset Preprocessing

In [4]:
X = dataset.drop(columns=['Agent'])
y = dataset['Agent']

In [5]:
def preprocess_independent_variables(X):
  preprocessed_X = pd.get_dummies(X, columns=['Agent_Type', 'Playstyle', 'Ability_Preference', 'Gun_Type'], drop_first=True)    # ONE HOT ENCODING
  ordinal_encoder = OrdinalEncoder(categories=[['Easy', 'Hard']])
  preprocessed_X['Difficulty'] = ordinal_encoder.fit_transform(preprocessed_X[['Difficulty']])    # ORDINAL ENCODING
  return preprocessed_X

def preprocess_dependent_variable(y):
  label_encoder = LabelEncoder()
  preprocessed_y = label_encoder.fit_transform(y)
  class_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
  print(class_mapping)
  return preprocessed_y

In [6]:
X

,Agent_Type,Playstyle,Difficulty,Ability_Preference,Gun_Type
0,Initiator,Info-gathering,Hard,Healing,SMGs
1,Controller,Balanced,Easy,Agility,Rifles
2,Controller,Aggressive,Hard,Agility,SMGs
3,Controller,Supportive,Easy,Smokes,Rifles
4,Sentinel,Map-control,Easy,Agility,Rifles
...,...,...,...,...,...
995,Initiator,Balanced,Easy,Agility,Machine Guns
996,Sentinel,Aggressive,Easy,Smokes,Shotguns
997,Initiator,Info-gathering,Easy,Smokes,SMGs
998,Sentinel,Supportive,Hard,Healing,Rifles


In [7]:
preprocessed_X = preprocess_independent_variables(X)
preprocessed_X

,Difficulty,Agent_Type_Duelist,Agent_Type_Initiator,Agent_Type_Sentinel,Playstyle_Balanced,Playstyle_Info-gathering,Playstyle_Map-control,Playstyle_Supportive,Ability_Preference_Flashes/Stuns,Ability_Preference_Healing,Ability_Preference_Information,Ability_Preference_Smokes,Gun_Type_Rifles,Gun_Type_SMGs,Gun_Type_Shotguns,Gun_Type_Snipers
0,1.0,False,True,False,False,True,False,False,False,True,False,False,False,True,False,False
1,0.0,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
2,1.0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,0.0,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False
4,0.0,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
996,0.0,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False
997,0.0,False,True,False,False,True,False,False,False,False,False,True,False,True,False,False
998,1.0,False,False,True,False,False,False,True,False,True,False,False,True,False,False,False


In [8]:
y

,Agent
0,Skye
1,Brimstone
2,Omen
3,Harbor
4,Killjoy
...,...
995,Sova
996,Killjoy
997,Fade
998,Sage


In [9]:
preprocessed_y = preprocess_dependent_variable(y)
preprocessed_y

{'Astra': 0, 'Breach': 1, 'Brimstone': 2, 'Chamber': 3, 'Clove': 4, 'Cypher': 5, 'Deadlock': 6, 'Fade': 7, 'Gekko': 8, 'Harbor': 9, 'Iso': 10, 'Jett': 11, 'Killjoy': 12, 'Neon': 13, 'Omen': 14, 'Phoenix': 15, 'Raze': 16, 'Reyna': 17, 'Sage': 18, 'Skye': 19, 'Sova': 20, 'Viper': 21, 'Vyse': 22, 'Yoru': 23}


array([19,  2, 14,  9, 12, 18, 10,  4, 12, 12,  3, 21,  9, 20, 22, 19, 17,
       12,  8, 14, 20, 21, 20, 18, 22, 14, 23, 10,  2, 21, 21, 16,  3, 16,
        8,  9, 20,  9,  5, 13,  8, 23,  4, 21, 19, 16, 15, 21, 16, 20, 19,
        8, 20,  5, 21, 19, 19, 19,  4, 14, 19, 20,  1, 21, 12, 16, 19, 23,
       21, 14,  9, 15,  1, 23, 23, 23, 20, 23,  3, 20, 10, 23,  1,  3, 17,
       12, 14,  4, 20, 20,  7,  7, 15, 14,  4,  5, 19,  9, 10, 12, 17, 18,
        4,  3,  9, 13, 13, 19, 23,  7, 19, 19,  1, 22, 18,  4,  6, 19, 23,
       18,  9,  8, 17, 12,  6, 19, 13, 10, 13, 20, 15,  9, 12,  5,  5,  3,
       16, 14, 21,  4, 18,  2, 20, 18, 19, 21, 18,  2, 22, 19, 16, 22, 17,
       14,  8, 15,  9,  2, 21,  8, 11, 11, 12, 23, 22, 22,  0, 18, 19,  8,
        7,  5, 19, 20,  7,  6,  7, 14, 19, 14,  6,  5,  4, 12,  2,  5, 19,
        9, 22, 11, 19, 13, 21, 17, 13,  5,  7, 17, 16, 18, 15, 21, 20, 14,
       18,  5, 18, 14, 20, 10, 15,  5, 23,  3, 16, 12, 14, 16, 20, 18, 19,
       18,  6, 12,  9,  4

## Train-Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_X, preprocessed_y, test_size = 0.2, random_state = 0)

In [11]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

800
800
200
200


## Model Initialization and Training

In [12]:
classifier = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    random_state=42
)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=42)

## Model Evaluation

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.89


In [15]:
report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         9
           5       0.83      1.00      0.91        10
           6       1.00      1.00      1.00         4
           7       1.00      0.89      0.94         9
           8       0.73      1.00      0.84         8
           9       0.92      0.86      0.89        14
          10       1.00      0.57      0.73         7
          11       0.90      0.90      0.90        10
          12       1.00      0.83      0.91        12
          13       0.40      1.00      0.57         2
          14       0.79      0.92      0.85        12
          15       0.75      0.50      0.60         6
          16       1.00      0.92      0.96        12
   

In [16]:
confusion = confusion_matrix(y_test, y_pred)
agents = y.unique().tolist()
confusion_with_labels = pd.DataFrame(confusion, index=agents, columns=agents)

# Adjust Pandas settings for full display
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Adjust width to avoid line breaks
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

print("Confusion Matrix:")
print()
confusion_with_labels

Confusion Matrix:



,Skye,Brimstone,Omen,Harbor,Killjoy,Sage,Iso,Clove,Chamber,Viper,Sova,Vyse,Reyna,Gekko,Yoru,Raze,Cypher,Neon,Phoenix,Breach,Fade,Deadlock,Jett,Astra
Skye,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
Brimstone,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Omen,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Harbor,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Killjoy,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Sage,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Iso,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Clove,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Chamber,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Viper,0,0,0,0,0,0,0,0,0,12,0,0,0,0,2,0,0,0,0,0,0,0,0,0


## Saving the Model

In [17]:
MODEL_STORE_PATH = f"/content/RandomForestClassifier_ValorantAgentPicker_accuracy_{int(accuracy*100)}.pkl"

In [18]:
with open(MODEL_STORE_PATH, 'wb') as file:
  pickle.dump(classifier, file)

## Testing the Model (Inference)

In [19]:
# GLOBALS.PY
PREPROCESSED_X_COLUMNS = [
    'Difficulty',
    'Agent_Type_Duelist',
    'Agent_Type_Initiator',
    'Agent_Type_Sentinel',
    'Playstyle_Balanced',
    'Playstyle_Info-gathering',
    'Playstyle_Map-control',
    'Playstyle_Supportive',
    'Ability_Preference_Flashes/Stuns',
    'Ability_Preference_Healing',
    'Ability_Preference_Information',
    'Ability_Preference_Smokes',
    'Gun_Type_Rifles',
    'Gun_Type_SMGs',
    'Gun_Type_Shotguns',
    'Gun_Type_Snipers'
]

DIFFICULTY_MAPPINGS = {
    'Easy': 0.0,
    'Hard': 1.0
}

AGENT_MAPPINGS = [
    'Astra', 'Breach', 'Brimstone', 'Chamber', 'Clove', 'Cypher',
    'Deadlock', 'Fade', 'Gekko', 'Harbor', 'Iso', 'Jett', 'Killjoy',
    'Neon', 'Omen', 'Phoenix', 'Raze', 'Reyna', 'Sage', 'Skye',
    'Sova', 'Viper', 'Vyse', 'Yoru'
]

In [20]:
def get_preprocessed_input(input_dict):
  example_data_preprocessed = pd.DataFrame([[''] * len(PREPROCESSED_X_COLUMNS)], columns=PREPROCESSED_X_COLUMNS)
  example_data_columns = PREPROCESSED_X_COLUMNS[:]

  for column in example_data_columns:
    for current_column, current_column_value in input_dict.items():
      if current_column in column:
        # ORDINAL COLUMNS
        if current_column == 'Difficulty':
          example_data_preprocessed[column] = DIFFICULTY_MAPPINGS[input_dict['Difficulty']]
        # ONE HOT ENCODING COLUMNS
        else:
          if current_column_value in column:
            example_data_preprocessed[column] = True
          else:
            example_data_preprocessed[column] = False
  return example_data_preprocessed

In [21]:
example_data = {
    'Agent_Type': 'Sentinel',
    'Playstyle': 'Aggressive',
    'Difficulty': 'Hard',
    'Ability_Preference': 'Agility',
    'Gun_Type': 'Sniper'
}

example_data_preprocessed = get_preprocessed_input(example_data)

with open(MODEL_STORE_PATH, 'rb') as file:
  classifier = pickle.load(file)

example_data_pred = classifier.predict(example_data_preprocessed)
print(AGENT_MAPPINGS[example_data_pred[0]])

Chamber
